In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option('display.max_rows', 268)
pd.set_option('display.max_columns', None)

In [2]:
%%time
path_events = '/Users/gabrieljnc/Desktop/GitHub/World_Cup_main/Data/raw_all_events_data.csv'
raw_data_events = pd.read_csv(path_events, low_memory=False)

CPU times: user 6.45 s, sys: 1.54 s, total: 7.99 s
Wall time: 8.3 s


In [3]:
%%time
path_matches = '/Users/gabrieljnc/Desktop/GitHub/World_Cup_main/Data/raw_matches_detail.csv'
raw_data_matches = pd.read_csv(path_matches)

CPU times: user 4.1 ms, sys: 7.23 ms, total: 11.3 ms
Wall time: 10.5 ms


In [4]:
# Calcular o uso de memória em bytes
memory_bytes = raw_data_events.memory_usage().sum()

# Converter para megabytes (opcional)
memory_mb = memory_bytes / 1024 / 1024

# Imprimir o tamanho total de memória em megabytes
print(f'O dataframe de eventos ocupa {memory_mb:.2f} megabytes de memória.')

O dataframe de eventos ocupa 500.35 megabytes de memória.


In [5]:
data_events_copy = raw_data_events.copy()
data_matches_copy = raw_data_matches.copy()

In [6]:
data_events = data_events_copy.loc[: , 
                         ['match_id', 'pass_shot_assist', 'period', 
                          'pass_body_part_name', 'shot_outcome_name', 'pass_type_name', 'type_name',
                          'possession_team_name', 'timestamp','pass_outcome_name', 'team_name',
                          'shot_type_name', 'shot_end_location', 'location', 
                          'pass_recipient_name', 'possession', 'position_name', 
                          'shot_statsbomb_xg', 'minute', 'pass_assisted_shot_id', 'id',
                          'player_name', 'shot_body_part_name', 'year', 
                          'pass_end_location', 'pass_goal_assist']]

data_matches = data_matches_copy.loc[: , ['match_id','home_team_home_team_name','home_score', 'away_team_away_team_name', 'away_score', 'match_date', 'competition_stage_name', 'stadium_country_name']]

In [7]:
data_matches.rename(columns = {'home_team_home_team_name': 'home_team_name', 'away_team_away_team_name': 'away_team_name', 'competition_stage_name':'stage', 'stadium_country_name':'world_cup'}, inplace = True)

In [8]:
columns_to_float = ['shot_end_location_x', 'shot_end_location_y','location_x', 
                    'location_y', 'pass_end_location_x', 'pass_end_location_y']

columns_to_category = [ 'shot_outcome_name', 'pass_type_name', 'type_name', 'pass_outcome_name', 
                        'shot_type_name','shot_body_part_name', 'position_name', 'team_name', 
                        'possession_team_name']

In [9]:
# Fuction that create two columns with x and y coordinates and return a dataframe with these 2 columns

def extract_coordinates(dataframe, column_name):
    """
    Extract x and y coordinates from a column in a pandas DataFrame.

    Args:
    dataframe: pandas DataFrame containing the column with the coordinates.
    column_name: name of the column to extract coordinates from.
    
    Returns:
    A new pandas DataFrame with two columns: '{column_name}_x' and '{column_name}_y',
    containing the x and y coordinates extracted from the original column.
    """
    extracted = dataframe[column_name].str.extract(r'\{(\d+\.\d+),(\d+\.\d+)\}')
    return pd.DataFrame({f'{column_name}_x':extracted[0], f'{column_name}_y': extracted[1]})

# Define a list of column names to extract coordinates from
columns_to_extract = ['shot_end_location', 'location', 'pass_end_location']

# Extract the coordinates for each column using list comprehension
extracted_dfs = [extract_coordinates(data_events, col) for col in columns_to_extract]

# Concatenate the resulting DataFrames
data_events = pd.concat([data_events] + extracted_dfs, axis=1)

# Drop the original columns
data_events.drop(columns=columns_to_extract, inplace=True)

In [10]:
def convert_column_type(dataframe, columns: list, dtype: str):
    """
    Convert columns in a pandas DataFrame to the specified data type.
    
    Args:
    dataframe: pandas DataFrame to be modified.
    columns: list of columns to be converted to the specified data type.
    dtype: the data type to which the columns will be converted. Valid values are 'float', 'int', 'category', 'bool'.
    
    Returns:
    None.
    """
    try:
        if dtype == 'float':
            dataframe[columns] = dataframe[columns].astype(float)
        elif dtype == 'int':
            dataframe[columns] = dataframe[columns].astype(int)
        elif dtype == 'category':
            for col in columns:
                dataframe[col] = dataframe[col].astype('category')
        else:
            raise ValueError(f"Invalid dtype: {dtype}. Valid values are 'float', 'int', 'category', 'bool'.")
    except KeyError as e:
        print(f"Error: column '{str(e)}' not found in DataFrame.")
    except ValueError as e:
        print(f"Error: {str(e)}")
    
    return None

In [11]:
convert_column_type(data_events, columns_to_float, 'float')
convert_column_type(data_events, columns_to_category, 'category')
data_matches['match_date'] = pd.to_datetime(data_matches['match_date'])
data_matches = data_matches.sort_values('match_date')

In [12]:
map_match_ids = {7525: 1, 7578: 2, 7577: 3, 7576: 4, 7529: 5, 7530: 6, 7531: 7, 7532: 8, 7535: 9, 7533: 10, 7534: 11, 7538: 12, 7537: 13, 7536: 14,
                 7539: 15, 7540: 16, 7541: 17, 7542: 18, 7544: 19, 7543: 20, 7545: 21, 7546: 22, 7547: 23, 7548: 24, 7549: 25, 7550: 26, 7552: 27, 
                 7551: 28, 7553: 29, 7556: 30, 7555: 31, 7554: 32, 7560: 33, 7557: 34, 7559: 35, 7558: 36, 7561: 37, 7563: 38, 7564: 39, 7562: 40, 
                 7565: 41, 7567: 42, 7568: 43, 7566: 44, 7572: 45, 7569: 46, 7571: 47, 7570: 48, 7580: 49, 7579: 50, 7582: 51, 7581: 52, 7584: 53, 
                 7583: 54, 7585: 55, 7586: 56, 8650: 57, 8649: 58, 8651: 59, 8652: 60, 8655: 61, 8656: 62, 8657: 63, 8658: 64, 3857286: 65, 3857282: 66, 
                 3857271: 67, 3857285: 68, 3857265: 69, 3857279: 70, 3857300: 71, 3857254: 72, 3857268: 73, 3857277: 74, 3857291: 75, 3857284: 76, 
                 3857258: 77, 3857290: 78, 3857298: 79, 3857287: 80, 3857301: 81, 3857272: 82, 3857273: 83, 3857274: 84, 3857288: 85, 3857297: 86, 
                 3857266: 87, 3857289: 88, 3857263: 89, 3857295: 90, 3857283: 91, 3857281: 92, 3857270: 93, 3857259: 94, 3857269: 95, 3857299: 96, 
                 3857267: 97, 3857261: 98, 3857278: 99, 3857294: 100, 3857260: 101, 3857275: 102, 3857264: 103, 3857257: 104, 3857296: 105, 3857292: 106, 
                 3857276: 107, 3857255: 108, 3857262: 109, 3857293: 110, 3857256: 111, 3857280: 112, 3869151: 113, 3869117: 114, 3869118: 115, 3869152: 116, 
                 3869253: 117, 3869219: 118, 3869254: 119, 3869220: 120, 3869420: 121, 3869321: 122, 3869486: 123, 3869354: 124, 3869519: 125, 3869552: 126, 
                 3869684: 127, 3869685: 128 }

map_players = {'Neymar da Silva Santos Junior':'Neymar', 'Thiago Emiliano da Silva':'Thiago Silva', 'Carlos Henrique Casimiro':'Casimiro',
                         'Philippe Coutinho Correia':'Philippe Coutinho', 'Marcos Aoás Corrêa':'Marquinhos', 'Danilo Luiz da Silva':'Danilo',
                         'Marcelo Vieira da Silva Júnior':'Marcelo', 'Éder Gabriel Militão':'Eder Militão', 'João Miranda de Souza Filho':'Miranda',
                         'Willian Borges da Silva':'Willian', 'Gabriel Fernando de Jesus':'Gabriel Jesus', 'Fágner Conserva Lemos':'Fágner',
                         'Lucas Tolentino Coelho de Lima':'Lucas Paquetá', 'José Paulo Bezzera Maciel Júnior':'Paulinho', 'Raphael Dias Belloli':'Raphinha',
                         'Alex Sandro Lobo Silva':'Alex Sandro', 'Vinícius José Paixão de Oliveira Júnior': 'Vinícius Júnior',
                         'Alisson Ramsés Becker':'Alisson Becker', 'Rodrygo Silva de Goes':'Rodrygo Goes', 'Antony Matheus dos Santos':'Antony', 
                         'Filipe Luís Kasmirsk':'Filipe Luís', 'Fernando Luiz Rosa':'Fernandinho', 'Richarlison de Andrade':'Richarlison', 
                         'Frederico Rodrigues Santos':'Fred','Daniel Alves da Silva':'Daniel Alves', 'Gleison Bremer Silva Nascimento':'Bremer', 
                         'Douglas Costa de Souza':'Douglas Costa', 'Roberto Firmino Barbosa de Oliveira':'Roberto Firmino', 
                         'Gabriel Teodoro Martinelli Silva':'Gabriel Martinelli','Bruno Guimarães Rodriguez Moura':'Bruno Guimarães', 
                         'Renato Soares de Oliveira Augusto':'Renato Augusto', 'Alex Nicolao Telles':'Alex Telles', 'Fábio Henrique Tavares':'Fabinho', 
                         'Pedro Guilherme Abreu dos Santos': 'Pedro','Ederson Santana de Moraes':'Ederson', 
                         'Éverton Augusto de Barros Ribeiro':'Éverton Ribeiro', 'Weverton Pereira da Silva':'Weverton', 'Lionel Andrés Messi Cuccittini': 'Lionel Messi', 'Kylian Mbappé Lottin': 'Kylian Mbappé',
                         'Ángel Fabián Di María Hernández': 'Ángel Di María'}

# Mapping new values for match_id
data_events['match_id'] = data_events['match_id'].replace(map_match_ids)
data_matches['match_id'] = data_matches['match_id'].replace(map_match_ids)

# Mapping new values for players
data_events['player_name'] = data_events['player_name'].replace(map_players)
data_events['pass_recipient_name'] = data_events['pass_recipient_name'].replace(map_players)

#Creating new columns
data_matches['match'] = data_matches['home_team_name'] + ' X ' + data_matches['away_team_name']
data_matches['final_score'] = data_matches['home_score'].astype(str) + ' - ' + data_matches['away_score'].astype(str)

In [13]:
def uso_de_memoria(dataframe): 
    # Calcular o uso de memória em bytes
    memory = dataframe.memory_usage().sum()

    # Converter para megabytes
    memory_mb = memory / 1024 / 1024

    # Imprimir o tamanho total de memória em megabytes
    return f"Dataframe ocupa {memory_mb:.2f} megabytes de memória."


tamanho_antes = uso_de_memoria(raw_data_events)
tamanho_depois = uso_de_memoria(data_events)

diferenca_percentual = ((float(tamanho_antes.split()[2]) - float(tamanho_depois.split()[2])) / float(tamanho_antes.split()[2])) * 100
print('-'*60)
print(f'Antes: {tamanho_antes}')
print('-'*60)
print(f'Depois: {tamanho_depois}')
print('-'*60)
print(f"A otimização reduziu o tamanho do dataframe em {diferenca_percentual:.2f}%")
print('-'*60)

------------------------------------------------------------
Antes: Dataframe ocupa 500.35 megabytes de memória.
------------------------------------------------------------
Depois: Dataframe ocupa 74.44 megabytes de memória.
------------------------------------------------------------
A otimização reduziu o tamanho do dataframe em 85.12%
------------------------------------------------------------


In [14]:
data_events.shape

(461845, 29)

In [16]:
csv_data_events = data_events.to_csv('../Data/main_data_all_events.csv',index = False)
csv_data_matches = data_matches.to_csv('../Data/main_data_matches_detail.csv',index = False)